# Import necessary libraries


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import re

pd.options.display.max_columns = 200
pd.options.display.max_rows = 3000

# Load the dataset 

In [ ]:
def parent_directory():

  # Create a relative path to the parent
  # of the current working directory
  relative_parent = os.path.join(os.getcwd(), os.pardir)
  # Return the absolute path of the parent directory
  return os.path.abspath(relative_parent)

print(parent_directory())

def str_2_int(value):
  if ',' in value:
    x = float(value.replace(',',''))
    return x
  else :
    return float(value)

salaries = pd.read_csv(parent_directory() + '/final_project/0.Ressources/05_2_salaries.csv')


# Renaming Columns and Adding Total Remuneration

In [ ]:


#Rename Column names because they were too long
salaries.columns = ['Time', 'Age', 'Industry', 'Job', 'Job Context', 'Salary', 'Bonus', 'Currency', 'Other Currency', 'Income Context', 'Country', 'US State', 'City', 'Experience', 'Field experience', 'Education', 'Gender', 'Race']

#Convert String to float
salaries["Salary"] = salaries["Salary"].apply(str_2_int)


#Replace NA values in Bonus by 0 and add a "Total Remuneration" Column
values = {"Bonus": 0.0}
salaries = salaries.fillna(value = values)
salaries["Total Remuneration"] = salaries["Salary"] + salaries["Bonus"]
move = salaries.pop("Total Remuneration")
salaries.insert(7,"Total Remuneration" ,move)


salaries


# Fixing Types

In [ ]:
salaries["Time"] = pd.to_datetime(salaries["Time"], format='%m/%d/%Y %H:%M:%S')

salaries.dtypes

# Check Missing Values

In [ ]:
pair_list = salaries["Currency"].unique()

In [ ]:
missing_values = salaries.isnull().sum()
missing_percentage = (missing_values / len(salaries)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_values, 'Percentage (%)': missing_percentage})
missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percentage (%)', ascending=False)

We can therefore see that over 99.2% of the dataset is payed in one of the currencies listed in the "Currency" column. For this reason we think it would make sense to remove the other currencies from our analysis as they would not have a significant impact on the results. We will therefore analyse salaries from the US, Great Britain, Canada, Euro Zone, Australia and New Zealand, Switzerland, South Africa, Sweden, Hong Kong and Japan. This may bias our views due to it being focused on advanced economies.   

# Get current exchange rate

In [ ]:
#We are going to use Selenium here to get the exchange rate of the currencies
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# instantiate a Chrome options object
options = webdriver.ChromeOptions()

# set the options to use Chrome in headless mode
options.add_argument("--headless=new")

# initialize an instance of the chrome driver (browser) in headless mode
driver = webdriver.Chrome(
    options=options,
)

# visit your target site
driver.get("https://www.oanda.com/currency-converter/live-exchange-rates/")

# extract all the product containers
rates = driver.find_elements(By.XPATH, "/html/body/div[1]/main/div/div/div[3]")

time.sleep(3)

In [ ]:
def get_current_rates():
    def get_rate(rate):
        if rate == "EUR":
          return 1
        #HKD doesnt exist unfortunately and I couldn't find it so i will hard code the rate
        if rate == "HKD":
          return 0.12
        
        #Users put AUD/NZD so we chose AUD as default 
        if rate == "AUD/NZD":
          rate = "AUD"
        
        rate_pair = rate + "/EUR"
        for rate in rates:
          if rate_pair in rate.text:
            exchange_rate_path = rate.text.split(rate_pair)[1].split("\n")
            return float(exchange_rate_path[1] + exchange_rate_path[2])

    pair_list = salaries['Currency'].unique().tolist()
    exchange_rate = {}
    
    for pair in pair_list:
      if pair == "Other":
        continue
      exchange_rate[pair] = get_rate(pair)
    
    return exchange_rate


def get_total_eur(row):
    for currency_key in ['Currency', 'Other Currency']:
        currency = row[currency_key]
        if currency in rates_list:
            return row['Total Remuneration'] * rates_list[currency]
    return None  # Return None if currency is not found

rates_list = get_current_rates()


In [ ]:

salaries['Total Remuneration EUR'] = salaries.apply(get_total_eur, axis=1)
salaries

# Check for Weird Values and Outliers

In [ ]:

year = salaries['Time'].dt.year
year.unique()
# Output 2021, 2022, 2023, 2024. Nothing weird

salaries['Job Context'].unique()
# Output is okay as well


salaries = salaries.drop(salaries[salaries['Total Remuneration'] == 0].index)
# 18 values that wasn't important for us as there is no remuneration. We can consider those as bad data

In [ ]:
# Function to detect outliers using IQR
def detect_outliers(dataframe, column):
    Q1 = dataframe[column].quantile(0.25)
    Q3 = dataframe[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    return dataframe[(dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)]

numerical_columns = salaries.select_dtypes(exclude=['object']).columns

# Detect outliers for numerical columns
outliers_data = {}
for column in numerical_columns:
    outliers = detect_outliers(salaries, column)
    outliers_data[column] = len(outliers)

# Display the count of outliers for each numerical column
outliers_data

Given the nature of this dataset, the presence of outliers in price-related columns (e.g., `Salary`, `Bonus`, `Total Remuneration`,`Total Remuneration EUR`) might be due to some salaries/bonus being higher than the others. Similarly, outliers in columns like `Time` are normal because it indicates all the timestamps

For the purpose of this project, we won’t remove these outliers since they can provide insights into this variety.

# Voici ton code intial sur currency rate conversion

In [ ]:
salaries["Other Currency"] = salaries['Other Currency'].fillna('NaN')
allowed_currencies = ['USD', 'GBP', 'CAD', 'EUR', 'AUD/NZD', 'CHF', 'ZAR', 'SEK', 'HKD', 'JPY']
other = ['Other', 'NaN', 'Na', 'N/a']
salaries = salaries[
    (salaries['Currency'].isin(allowed_currencies) & salaries['Other Currency'].isin(other) ) | (salaries['Currency'].isin(other) & salaries['Other Currency'].isin(allowed_currencies))]
print(salaries['Currency'].unique())
print(salaries['Other Currency'].unique())
len(salaries)


# prompt: make a function that converts the values of the "Total Remuneration" into EUR depending on the currency located in "Currency" or "Other currency" with current exchange rates

def convert_to_eur(row):
  """Converts the 'Total Remuneration' to EUR based on the currency in 'Currency' or 'Other Currency'.

  Args:
    row: A pandas Series representing a row in the DataFrame.

  Returns:
    The 'Total Remuneration' in EUR.
  """
  #Current Exchange Rates
  exchange_rates = {
      'USD': 0.93,  # EUR/USD rate
      'GBP': 1.15,  # EUR/GBP rate
      'CAD': 0.68,  # EUR/CAD rate
      'EUR': 1.00,  # EUR/EUR rate
      'AUD/NZD': 0.60,  # EUR/AUD rate (assuming the same for NZD for simplicity)
      'CHF': 0.95,  # EUR/CHF rate
      'ZAR': 0.05,  # EUR/ZAR rate
      'SEK': 0.09,  # EUR/SEK rate
      'HKD': 0.12,  # EUR/HKD rate
      'JPY': 0.0071,  # EUR/JPY rate
  }

  if row['Currency'] in exchange_rates:
    currency = row['Currency']
  elif row['Other Currency'] in exchange_rates:
    currency = row['Other Currency']
  else:
    return None  # Return None if currency is not found

  return row['Total Remuneration'] * exchange_rates[currency]

# Apply the function to create a new column
salaries['Total Remuneration EUR'] = salaries.apply(convert_to_eur, axis=1)
salaries

# Questions
1. Which industry pays the most?
2. How does salary increase given years of experience?
3. How do salaries compare for the same role in different locations?
4. How much do salaries differ by gender and years of experience?
5. How do factors like race and education level correlate with salary?
6. Is there a “sweet spot” total work experience vs years in the specific field?

In [ ]:
salary_average = salaries[['Industry','Total Remuneration EUR']]
list_pay_industry = salary_average.groupby("Industry").mean()
print(list_pay_industry.sort_values(by="Total Remuneration EUR", ascending= False))

